In [1]:
import numpy as np
import torch

# Environment + utilities
from helpers_HW.HW3.ion_trap import IonTrapEnv
from helpers_HW.HW3.utils import is_valid_srv

# Your agent
from model1 import model  # or from model import model


In [2]:
import sys
print(sys.executable)


/opt/homebrew/opt/python@3.11/bin/python3.11


In [3]:
print("Torch version:", torch.__version__)

# Create env just to test
env = IonTrapEnv(
    phases={
        'pulse_angles': [np.pi/2],
        'pulse_phases': [np.pi/2],
        'ms_phases': [-np.pi/2]
    },
    num_ions=3,
    goal=[[1, 1, 1]]
)

print("Environment OK")


Torch version: 2.9.0
Environment OK


In [4]:
# Check valid SRVs

from itertools import product

all_srv = list(product([1,2,3], repeat=3))
valid_srv = torch.tensor(
    [srv for srv in all_srv if srv == (1,1,1) or is_valid_srv(srv)[0]]
)

valid_srv


tensor([[1, 1, 1],
        [1, 2, 2],
        [2, 1, 2],
        [2, 2, 1],
        [2, 2, 2],
        [2, 2, 3],
        [2, 3, 2],
        [2, 3, 3],
        [3, 2, 2],
        [3, 2, 3],
        [3, 3, 2],
        [3, 3, 3]])

In [ ]:
# Instantiate agent

# First time no training results. Run train1.py -> get file qnet.pt -> will be used in test of agent for SRVs
# Must include ion_trap, model and qnet.pt in submission zip? 

agent = model()
print("Agent initialized")



Agent initialized


In [6]:
# Test agent on one SRV

test_srv = torch.tensor([[3, 3, 3]])

preds = agent.pred(test_srv)
preds


/Users/synneeikeland/Documents/ML_course/ML4Physics_homework3/model1.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(srv, dtype=torch.float32),


[tensor([0, 6, 4, 5, 1, 5, 3, 6, 3, 3])]

In [7]:
# Verify environment reward for that sequence

srv = test_srv[0]

env = IonTrapEnv(
    phases={
        'pulse_angles': [np.pi/2],
        'pulse_phases': [np.pi/2],
        'ms_phases': [-np.pi/2]
    },
    num_ions=3,
    goal=[list(srv)],
    max_steps=10
)

state = env.reset()
reward = 0

for action in preds[0]:
    state, reward, done = env.step(int(action))
    if done:
        break

print("Final reward:", reward)
print("Sequence length:", len(preds[0]))


Final reward: 0.0
Sequence length: 10


In [8]:
# Test agent on all valid SRVs

preds = agent.pred(valid_srv)

lengths = []
success = []

for srv, seq in zip(valid_srv, preds):
    env = IonTrapEnv(
        phases={
            'pulse_angles': [np.pi/2],
            'pulse_phases': [np.pi/2],
            'ms_phases': [-np.pi/2]
        },
        num_ions=3,
        goal=[list(srv)],
        max_steps=10
    )

    env.reset()
    reward = 0
    for a in seq:
        _, reward, done = env.step(int(a))
        if done:
            break

    lengths.append(len(seq))
    success.append(reward)

print("Average sequence length:", np.mean(lengths))
print("Success rate:", np.mean(success))


Average sequence length: 5.916666666666667
Success rate: 0.6666666666666666


In [9]:
# Debug state representation

srv = torch.tensor([2,2,2])
env = IonTrapEnv(
    phases={
        'pulse_angles': [np.pi/2],
        'pulse_phases': [np.pi/2],
        'ms_phases': [-np.pi/2]
    },
    num_ions=3,
    goal=[list(srv)]
)

state = env.reset()
print("State shape:", np.array(state).shape)
print("First amplitudes:", state[:5])


State shape: (27, 1)
First amplitudes: [[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
